In [1]:
import os
from pydub import AudioSegment
from tqdm import tqdm
import librosa
import numpy as np
import soundfile as sf

/Users/yukewei/anaconda3/envs/SProject/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
KEYFOBFOLDER = "/Users/yukewei/SP-testing/HackRFOneSignal/KeyFobSignal"
ATTACKERFOLDER = "/Users/yukewei/SP-testing/HackRFOneSignal/AttackerSignal"
GENERATEDFOLDER = "/Users/yukewei/SP-testing/Generated Signal"
input_data_1 = "/Users/yukewei/SP-testing/Machine Learning Model/CNN/input_data_1"
input_data_0 = "/Users/yukewei/SP-testing/Machine Learning Model/CNN/input_data_0"

Preprocessing will do two function:
1. check the signals sizes are 600000(sample rate * signal duration = 2000000 * 0.3 )
2. change the signals from stereo to mono if needed

In [3]:
def check_300ms(folder_path):
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {folder_path}"):
        if filename.endswith(".wav"):
            file_path = os.path.join(folder_path, filename)
            
            # Load the audio using librosa
            audio, sample_rate = librosa.load(file_path, sr=None, mono=True)
            
            # Check if the audio is shorter than 600,000 samples
            if len(audio) < 600000:
                pad_length = 600000 - len(audio)
                # Pad the audio with zeros to reach the desired length
                audio = np.pad(audio, (0, pad_length), mode='constant')
                
                # Overwrite the original audio file
                sf.write(file_path, audio, sample_rate, subtype='PCM_16')

In [4]:
# Function to convert audio to mono and save
def convert_to_mono_and_save(input_folder, output_folder):
    for filename in tqdm(os.listdir(input_folder), desc=f"Processing {input_folder}"):
        if filename.endswith(".wav"):
            file_path = os.path.join(input_folder, filename)
            num_channels = AudioSegment.from_file(file_path).channels

            # Convert to mono audio if not already mono
            audio = AudioSegment.from_file(file_path, format="wav")
            if num_channels > 1:
                audio = audio.set_channels(1)
                
            output_file = os.path.join(output_folder, f"mono_{filename}")
            audio.export(output_file, format="wav")

In [5]:
def process_audio_folder(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process audio files in the input folder
    check_300ms(input_folder)
    convert_to_mono_and_save(input_folder, output_folder)

In [6]:
# Process audio files for Class 1
process_audio_folder(KEYFOBFOLDER, input_data_1)

# Process audio files for Class 0
process_audio_folder(ATTACKERFOLDER, input_data_0)
process_audio_folder(GENERATEDFOLDER, input_data_0)

Processing /Users/yukewei/SP-testing/HackRFOneSignal/KeyFobSignal:   0%|          | 0/2664 [00:00<?, ?it/s]

Processing /Users/yukewei/SP-testing/HackRFOneSignal/KeyFobSignal: 100%|██████████| 2664/2664 [00:20<00:00, 132.57it/s]
Processing /Users/yukewei/SP-testing/HackRFOneSignal/KeyFobSignal: 100%|██████████| 2664/2664 [00:06<00:00, 386.27it/s]
Processing /Users/yukewei/SP-testing/HackRFOneSignal/AttackerSignal: 100%|██████████| 1418/1418 [00:03<00:00, 375.46it/s]
Processing /Users/yukewei/SP-testing/HackRFOneSignal/AttackerSignal: 100%|██████████| 1418/1418 [00:01<00:00, 893.03it/s]
Processing /Users/yukewei/SP-testing/Generated Signal: 100%|██████████| 1/1 [00:00<00:00, 38479.85it/s]
Processing /Users/yukewei/SP-testing/Generated Signal: 100%|██████████| 1/1 [00:00<00:00, 33288.13it/s]
